<a href="https://colab.research.google.com/github/rwcitek/C11-capstone-project/blob/main/eBird/ebird_bigquery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with BigQuery


We'll see if this runs in CoLab.




### Set up the workspace


In [1]:
!rm -rf data/


In [2]:
mkdir data

In [3]:
cd data

/content/data


In [4]:
pwd

'/content/data'

### Download some sample data

In [5]:
%%capture output
%%bash
apt-get update
apt-get install -y tree jq curl less

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [631 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,456 kB]
Get:13 http://archive.ubuntu.com/

In [6]:
!curl -O 'https://ebird.org/downloads/samples/ebd-datafile-SAMPLE.zip'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  455k  100  455k    0     0   792k      0 --:--:-- --:--:-- --:--:--  792k


In [7]:
!ls -la


total 464
drwxr-xr-x 2 root root   4096 Nov 21 23:10 .
drwxr-xr-x 1 root root   4096 Nov 21 23:10 ..
-rw-r--r-- 1 root root 466849 Nov 21 23:10 ebd-datafile-SAMPLE.zip


In [8]:
!unzip ebd-datafile-SAMPLE.zip

Archive:  ebd-datafile-SAMPLE.zip
   creating: ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/
  inflating: __MACOSX/._ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE  
  inflating: ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/ebd_US-AL-101_202204_202204_relApr-2022.txt  
  inflating: __MACOSX/ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/._ebd_US-AL-101_202204_202204_relApr-2022.txt  
  inflating: ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/terms_of_use.txt  
  inflating: __MACOSX/ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/._terms_of_use.txt  
  inflating: ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/BCRCodes.txt  
  inflating: __MACOSX/ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/._BCRCodes.txt  
  inflating: ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/eBird_Basic_Dataset_Metadata_v1.14.pdf  
  inflating: __MACOSX/ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/._eBird_Basic_Dataset_Metadata_v1.14.pdf  
  inflating: ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/IBACod

In [9]:
!ls -la


total 472
drwxr-xr-x 4 root root   4096 Nov 21 23:10 .
drwxr-xr-x 1 root root   4096 Nov 21 23:10 ..
-rw-r--r-- 1 root root 466849 Nov 21 23:10 ebd-datafile-SAMPLE.zip
drwxr-xr-x 2 root root   4096 Jun  8  2022 ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE
drwxr-xr-x 3 root root   4096 Nov 21 23:10 __MACOSX


In [10]:
!rm -rf __MACOSX/

In [11]:
!tree ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/


ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/
├── BCRCodes.txt
├── ebd_US-AL-101_202204_202204_relApr-2022.txt
├── eBird_Basic_Dataset_Metadata_v1.14.pdf
├── IBACodes.txt
├── recommended_citation.txt
├── terms_of_use.txt
└── USFWSCodes.txt

0 directories, 7 files


In [12]:
%%bash
wc -l ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/ebd_US-AL-101_202204_202204_relApr-2022.txt


1400 ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/ebd_US-AL-101_202204_202204_relApr-2022.txt


## Set up Google Cloud Platform

In [15]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [73]:
import os
project_id = "default-256400"
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id


## Need to create a *bucket*

Go to https://console.cloud.google.com/storage/browser




Got a bucket:

- https://console.cloud.google.com/storage/browser/c11-ebird;tab=objects?project=default-256400&prefix=&forceOnObjectsSortingFiltering=false

## Phase 1:



1. upload the CSV file to GS
1. import into BQ



In [21]:
file = 'ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/ebd_US-AL-101_202204_202204_relApr-2022.txt'


In [22]:
!gcloud storage cp {file} gs://c11-ebird/

Copying file://ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/ebd_US-AL-101_202204_202204_relApr-2022.txt to gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt


In [26]:
!gcloud storage ls -l gs://c11-ebird/

    504698  2023-11-21T23:28:26Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt
TOTAL: 1 objects, 504698 bytes (492.87kiB)


### From web interface

Create a data set and import CSV to data set:

- https://cloud.google.com/bigquery/docs/loading-data-cloud-storage-csv

### Repeat using bq command

In [86]:
!bq ls {project_id}:ebird

    tableId     Type    Labels   Time Partitioning   Clustered Fields  
 ------------- ------- -------- ------------------- ------------------ 
  ebird_small   TABLE                                                  


In [76]:
!bq rm -f {project_id}:ebird.ebird_small_2

In [74]:
%%bash
bq --project_id=${GOOGLE_CLOUD_PROJECT} \
  load \
  --source_format=CSV \
  --field_delimiter='\t' \
  --autodetect \
  --skip_leading_rows=1 \
  ebird.ebird_small_2 \
  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt


Waiting on bqjob_r2c10e32fbe8bf2d0_0000018bf481da69_1 ... (1s) Current status: DONE   


In [75]:
!bq ls {project_id}:ebird

     tableId      Type    Labels   Time Partitioning   Clustered Fields  
 --------------- ------- -------- ------------------- ------------------ 
  ebird_small     TABLE                                                  
  ebird_small_2   TABLE                                                  


## Phase 2:


1. compress the CSV file with gzip
1. upload the compressed CSV file to GS
1. import into BQ



In [78]:
!ls -la {file}

-rw-r--r-- 1 root root 504698 May 19  2022 ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/ebd_US-AL-101_202204_202204_relApr-2022.txt


In [79]:
!gzip {file}

In [81]:
!ls -la {file}.gz


-rw-r--r-- 1 root root 60011 May 19  2022 ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/ebd_US-AL-101_202204_202204_relApr-2022.txt.gz


In [82]:
!gcloud storage cp {file}.gz gs://c11-ebird/

Copying file://ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/ebd_US-AL-101_202204_202204_relApr-2022.txt.gz to gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.gz


In [83]:
!gcloud storage ls -l gs://c11-ebird/

    504698  2023-11-21T23:28:26Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt
     60011  2023-11-22T00:57:52Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.gz
TOTAL: 2 objects, 564709 bytes (551.47kiB)


In [87]:
!bq ls {project_id}:ebird

    tableId     Type    Labels   Time Partitioning   Clustered Fields  
 ------------- ------- -------- ------------------- ------------------ 
  ebird_small   TABLE                                                  


In [88]:
!bq rm -f {project_id}:ebird.ebird_small_3

In [89]:
%%bash
bq --project_id=${GOOGLE_CLOUD_PROJECT} \
  load \
  --source_format=CSV \
  --field_delimiter='\t' \
  --autodetect \
  --skip_leading_rows=1 \
  ebird.ebird_small_3 \
  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.gz


Waiting on bqjob_r610c656fefcded4c_0000018bf48bc0f5_1 ... (1s) Current status: DONE   


In [90]:
!bq ls {project_id}:ebird

     tableId      Type    Labels   Time Partitioning   Clustered Fields  
 --------------- ------- -------- ------------------- ------------------ 
  ebird_small     TABLE                                                  
  ebird_small_3   TABLE                                                  


In [91]:
!bq rm -f {project_id}:ebird.ebird_small_3

In [92]:
!bq ls {project_id}:ebird

    tableId     Type    Labels   Time Partitioning   Clustered Fields  
 ------------- ------- -------- ------------------- ------------------ 
  ebird_small   TABLE                                                  


## Phase 3:


1. compress the CSV file with gzip
1. create a tar file with the compressed CSV file
1. untar and stream the compressed CSV file to GS
1. import into BQ



## Phase 4:



1. compress the CSV file with gzip
1. create a tar file with the compressed CSV file
1. upload the tar file to DO's object store
1. get, untar, and stream the compressed CSV file to GS
1. import into BQ
